## User Satisfaction Analysis

In [103]:
# import required packages and modules
import sys
sys.path.insert(0,'../scripts/')

import pandas as pd
import numpy as np
from numpy import percentile

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# 
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.cluster import KMeans

# save model
import pickle

#
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go  
import matplotlib.pyplot as plt
from IPython.display import Image
from plotly.subplots import make_subplots
# 
from scipy.spatial.distance import cdist
from sklearn.preprocessing import StandardScaler, normalize

In [104]:
# load the required datasets
df = pd.read_csv('../data/cleaned_telecom_data_source.csv', index_col=0)
user_engagement_df  = pd.read_csv('../data/user_engagement.csv', index_col=0)
user_experience_df = pd.read_csv('../data/user_experience.csv', index_col=0)

In [105]:

user_engagement_df.info()


<class 'pandas.core.frame.DataFrame'>
Float64Index: 106856 entries, 33601001722.0 to 882397108489451.0
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Cluster            106856 non-null  int64  
 1   Session Frequency  106856 non-null  int64  
 2   Duration           106856 non-null  float64
 3   Total Data Usage   106856 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 4.1 MB


In [106]:
user_experience_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 106856 entries, 33601001722.0 to 882397108489451.0
Data columns (total 4 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Cluster                     106856 non-null  int64  
 1   Total Avg RTT (ms)          106856 non-null  float64
 2   Total Avg Bearer TP (kbps)  106856 non-null  float64
 3   Total Avg TCP (Bytes)       106856 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 4.1 MB


In [107]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df.head(1)

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),250 Kbps < DL TP < 1 Mbps (%),DL TP > 1 Mbps (%),UL TP < 10 Kbps (%),10 Kbps < UL TP < 50 Kbps (%),50 Kbps < UL TP < 300 Kbps (%),UL TP > 300 Kbps (%),Activity Duration DL (ms),Activity Duration UL (ms),Dur. (ms).1,Handset Manufacturer,Handset Type,Nb of sec with Vol DL < 6250B,Nb of sec with Vol UL < 1250B,Social Media DL (Bytes),Social Media UL (Bytes),Google DL (Bytes),Google UL (Bytes),Email DL (Bytes),Email UL (Bytes),Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes),Total Data (Bytes),Social Media (Bytes),Google (Bytes),Youtube (Bytes),Netflix (Bytes),Gaming (Bytes),Email (Bytes),Other (Bytes),Social Media (MB),Google (MB),Youtube (MB),Netflix (MB),Gaming (MB),Email (MB),Other (MB),Total Data (MB)
0,1.311448e+19,2019-04-04 12:01:00,770.0,2019-04-25 14:35:00,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,42.0,5.0,23.0,44.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,37624.0,38787.0,1.823653e+09,Samsung,Samsung Galaxy A5 Sm-A520F,213.0,214.0,1545765.0,24420.0,1634479.0,1271433.0,3563542.0,137762.0,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0,345629377.0,1570185.0,2905912.0,18355943.0,17855187.0,292426453.0,3701304.0,180558843.0,1.570185,2.905912,18.355943,17.855187,292.426453,3.701304,180.558843,345.629377


In [108]:
# 
user_engagement_df.head(3)

,Cluster,Session Frequency,Duration,Total Data Usage
User Id,,,,
3.360100e+10,3,1,116720.0,878690574.0
3.360100e+10,0,1,181230.0,156859643.0
3.360100e+10,0,1,134969.0,595966483.0


In [109]:
# 
user_experience_df.head(1)

,Cluster,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total Avg TCP (Bytes)
MSISDN/Number,,,,
3.360100e+10,1,92.0,76.0,0.0


### Task 4. 1 - Write a python program to assign:
* engagement score to each user. Consider the engagement score as the Euclidean distance between the user data point & the less engaged cluster (use the first clustering for this) (Euclidean Distance)
* experience score to each user. Consider the experience score as the Euclidean distance between the user data point & the worst experience’s cluster. 


#### 4.1.1 Assign engagement score to each user

In [110]:
with open("../models/user_engagement.pkl", "rb") as f:
    kmeans1 = pickle.load(f)
less_engaged_score = 3

In [112]:
engagement_df = user_engagement_df.set_index('Cluster')[
    ['Duration', 'Total Data Usage', 'Session Frequency']]
scaler = StandardScaler()
scaled_array = scaler.fit_transform(engagement_df)
pd.DataFrame(scaled_array).head(5)

<class 'pandas.core.frame.DataFrame'>
Float64Index: 106856 entries, 33601001722.0 to 882397108489451.0
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Cluster            106856 non-null  int64  
 1   Session Frequency  106856 non-null  int64  
 2   Duration           106856 non-null  float64
 3   Total Data Usage   106856 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 4.1 MB


,0,1,2
0,-0.103987,0.107641,-0.120240
1,0.118685,-0.317516,-0.120240
2,-0.040996,-0.058883,-0.120240
3,-0.334708,-0.161160,-0.120240
4,-0.378800,0.448506,0.177556


In [ ]:
data_normalized = normalize(scaled_array)
pd.DataFrame(data_normalized).head(5)
distance = kmeans1.fit_transform(data_normalized)
distance_from_less_engaged_score = list(
    map(lambda x: x[less_engaged_score], distance))
user_engagement_df['Engagement_Score'] = distance_from_less_engaged_score
user_engagement_df.head(5)

#### 4.1.2 Assign experience score to each user

In [114]:
with open("../models/user_experience.pkl", "rb") as f:
    kmeans2 = pickle.load(f)
worst_experience = 0

In [118]:
experience_df = user_experience_df.set_index('Cluster')[
    ['Total Avg RTT (ms)', 'Total Avg Bearer TP (kbps)', 'Total Avg TCP (Bytes)']]	
scaler = StandardScaler()
scaled_array = scaler.fit_transform(experience_df)
pd.DataFrame(scaled_array).head(5)

,0,1,2
0,-0.081530,-0.302769,-0.077651
1,-0.095825,-0.302439,-0.077651
2,-0.082423,-0.302468,-0.077651
3,-0.060980,-0.300299,-0.077644
4,-0.020775,0.512609,-0.017382


In [121]:
data_normalized = normalize(scaled_array)
pd.DataFrame(data_normalized).head(5)
experience_df = user_experience_df.set_index('Cluster')
distance = kmeans2.fit_transform(data_normalized)
distance_from_worst_experience = list(
    map(lambda x: x[worst_experience], distance))
user_experience_df['Experience_Score'] = distance_from_worst_experience
user_experience_df.head(5)

,Cluster,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total Avg TCP (Bytes),Experience_Score
MSISDN/Number,,,,,
3.360100e+10,1,92.0,76.0,0.0,1.823791
3.360100e+10,1,60.0,99.0,0.0,1.815926
3.360100e+10,1,90.0,97.0,0.0,1.823238
3.360101e+10,1,138.0,248.0,2132.0,1.833242
3.360101e+10,0,228.0,56844.0,18699260.0,0.176464


### Task 4.2 - Consider the average of both engagement & experience scores as  the satisfaction score & report the top 10 satisfied customer 

### Task 4.3 - Build a regression model of your choice to predict the satisfaction score of a customer. 

### Task 4.4 - Run a k-means (k=2) on the engagement & the experience score . 

### Task 4.5 - Aggregate the average satisfaction & experience score per cluster. 

### Task 4.6 - Export your final table containing all user id + engagement, experience & satisfaction scores in your local MySQL database. Report a screenshot of a select query output on the exported table. 